# Library Documentation

See the glossary [here](../models/README.md#Glossary) for documentation references.

# Connectivity Cost Estimation

Additional documentation for **[Scenario Types](#Scenarios)** and **[Parameters and Cost Drivers](#Parameters)**.

This notebook provides an modeling interface for estimating the total cost of connecting a group of schools.
The process is split into the following categories:
* **Configuration**: where you can configure a variety of model parameters
* **Running models**: where you can start the model execution
* **Outputs**: where you can examine the results in tables and plots and download them locally

Note that the notebook is interactive.
Namely, you can update the configurations, run the models again, and generate a new set of outputs.
You will need to explicitly re-run the models again after a re-configuration (by clicking the `Run Model` button), and you will need to explicitly re-generate the outputs (by clicking the various `Generate ...` buttons) to see the ouputs updated.

You can find a brief description of the model parameters [below](#Parameters). The interactive dashboard below allows you to configure the scenario of interest and the cost models being run inside that scenario.

If you would like to upload a configuration from a local machine, use the `Import Config` button below.

In [ ]:
import json
import os
from ipywidgets import FileUpload, Output, Label

from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput

import warnings
warnings.filterwarnings('ignore')


show_map = os.environ.get("FF_STATIC_MAP_NOTEBOOK", False)

output = Output()

upload = FileUpload(accept='.json', multiple=True, description="Import Config")
inputs = CostEstimationParameterInput(local_data_workspace='sample_workspace', show_map=show_map)

def on_upload_change(change):
    try:
        imported = json.loads(upload.value[0].content.tobytes())
    except IndexError:
        imported = {}
    with output:
        output.clear_output()
        inputs.update(imported)
    dispaly(imported)
    display(upload, inputs.parameter_input(), output)

upload.observe(on_upload_change, names='value')
display(upload, inputs.parameter_input(), output)

## Running the Model

Runs the model and displays summary tables that show aggregated cost statistics across the schools of interest.

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import Button, Output
from tqdm import tqdm
from IPython.display import clear_output
import sys
import time
import logging
import geopandas as gpd

from giga.models.scenarios.scenario_dispatcher import create_scenario
from giga.data.space.model_data_space import ModelDataSpace
from giga.schemas.output import OutputSpace
from giga.viz.notebooks.helpers import run_message
from giga.utils.logging import LOGGER
from giga.viz.notebooks.helpers import download_link_scenario_config
from giga.viz.notebooks.tables import display_summary_table, plot_cost_breakdown
from giga.viz.notebooks.helpers import output_to_table, download_link_frame

button = Button(description="Run Model")
output = Output()

output_space = None
data_space = None
config = None

display(button, output)

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        
        global config
        global data_space
        global output_space
        
        verbose = inputs.dashboard_parameters()["verbose"]
        LOGGER.propagate = verbose
        
        config = inputs.scenario_parameters()
        data_space = ModelDataSpace(inputs.data_parameters())
        output_space = OutputSpace()
        scenario = create_scenario(config, data_space, output_space)
        
        output_space = scenario.run(progress_bar=verbose)
        run_message()
        
        display(download_link_scenario_config(inputs))
        
        display_summary_table(output_space)
        border_file = 'parameter_workspace/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp'
        borders = gpd.read_file(border_file)
        borders['NAME'] = borders['NAME'].apply(lambda x: x.lower())
        cid = inputs.data_parameters().school_data_conf.country_id
        country_id = 'rwanda' if cid != 'brazil' else cid
        b = borders[borders["NAME"] == country_id]
        plot_cost_breakdown(output_space, data_space, b)
        
        table = output_to_table(output_space)
        display(download_link_frame(table))

button.on_click(on_button_clicked)

## Result Maps

The maps below show electricity avaibility and costs.


In [ ]:
import geopandas as gpd
from ipywidgets import HTML

from giga.viz.notebooks.tables import display_summary_table, output_to_capex_details, plot_cost_breakdown
from giga.viz.notebooks.helpers import output_to_table, button_cb, download_link_frame
from giga.viz.notebooks.maps import show_cost_map, show_electricity_map
from giga.viz.notebooks.fiber import plot_fiber_connections, default_rwanda_map


def action():
    
    if inputs.data_parameters().school_data_conf.country_id == 'brazil':
        display(HTML("<hr><b><font color='#5b8ff0'>Unable to display maps for Brazil</b><br><br>"))
        return
    
    display(HTML("<hr><b><font color='#5b8ff0'>Electricity Map</b><br><br>"))
    display(show_electricity_map(data_space))
    
    display(HTML("<hr><b><font color='#5b8ff0'>Cost Map</b><br><br>"))
    display(show_cost_map(data_space, output_space))
    
    if output_space.fiber_costs is not None:
        display(HTML("<hr><b><font color='#5b8ff0'>Fiber Map</b><br><br>"))
        display(plot_fiber_connections(data_space.fiber_coordinates, data_space.school_coordinates, output_space.fiber_costs.technology_results.distances, m=default_rwanda_map()))
        
    
button, click = button_cb("Show Maps", action)
button.on_click(click)

The table below shows the complete model outputs by school.

In [ ]:
from IPython.display import display, HTML
from giga.viz.notebooks.helpers import output_to_table, button_cb


def action():
    table = output_to_table(output_space)
    display(HTML('<h2>Cost Table</h2>'), table)
    
button, click = button_cb("Show Full Table", action)
button.on_click(click)

## Scenarios

The following scenarios are available in the model:
* `Lowest Cost`: determines the lowest cost of connectivity for the schools in question by selecting across the available and feasible technologies
* `Budget Constrained`: computes the minimum cost under a constrained budget. When this scenario is selected, the models will attempt to connect the maximum number of schools within the given budget. The budget constraint is set with respect to the total NPV of the schools in question
* `Fiber Only`: computes the total cost of connecting all the unconnected schools with fiber technology
* `Satellite Only`: computes the total cost of connecting all the unconnected schools with satellite technology
* `Cellular Only`: computes the total cost of connecting all the unconnected schools with cellular technology
* `P2P Only`: computes the total cost of connecting all the unconnected schools with P2P technology

## Parameters

The following parameters can be configured in the model:

* **Scenario**
    * `Cost Scenario` determines which scenario to estimate costs for; either `Minimum Cost` which finds the cheapest technology can be selected, or an individual technology can be selected which will find the costs for just that technology
    * `OpEx Years` determines the number of years that will be considered in the total cost estimates, where total cost is CapEx + OpEx * `OpEx Years`
    * `Bandwidth Demand (Mbps)` determines the expected demand at each school being considered
    * `Project Budget (Millions USD)` sets the maximum budget for the connectivity project being analyzed, this budget is for the NPV of the project
* **Fiber Model**
    * `Annual cost per Mbps (USD)`: the annual cost of connectivity per Mbps in US Dollars
    * `Cost Per km (USD)` is the average cost of laying fiber lines per km in US Dollars
    * `Maintenance Cost per km (USD)` is the expected annual maintenace cost of new fiber lines in US Dollars
    * `Maximum Connection Length (km)` is the maximum length of an individual fiber connection, if a single fiber connection exceeds this length, it will not be considered feasible
    * `Annual Power Required (kWh)` is the annual power in kWh needed to operate the equipment
    * `Economies of Scale` indicates if an economies of scale approach should be used when estimating the needed length of fiber lines
* **Satellite Model**
    * `Installation Cost (USD)` is the cost of intalling only the technology equipment (no electricity) at the school site
    * `Annual cost per Mbps (USD)`is the annual cost of connectivity per Mbps in US Dollars
    * `Annual Maintenance Cost (USD)` is the annual cost of maintaining the  equipment at the school site
    * `Annual Power Required (kWh)` is the annual power in kWh needed to operate the equipment
* **Cellular Model**
    * `Installation Cost (USD)` is the cost of intalling only the technology equipment (no electricity) at the school site
    * `Annual cost per Mbps (USD)`is the annual cost of connectivity per Mbps in US Dollars
    * `Annual Maintenance Cost (USD)` is the annual cost of maintaining the  equipment at the school site
    * `Annual Power Required (kWh)` is the annual power in kWh needed to operate the equipment
    * `Maximum Cell Tower Range (km)` is the maximum distance from a cell tower that a school can receive internet service
* **P2P Model**
    * `Installation Cost (USD)` is the cost of intalling only the technology equipment (no electricity) at the school site
    * `Annual cost per Mbps (USD)`is the annual cost of connectivity per Mbps in US Dollars
    * `Annual Maintenance Cost (USD)` is the annual cost of maintaining the  equipment at the school site
    * `Annual Power Required (kWh)` is the annual power in kWh needed to operate the equipment
    * `Maximum  Range (km)` is the maximum distance from a cell tower that a school can receive internet service
* **Electricity Model**
    * `Cost per kWh (USD)` is the expected average cost of electricity for the schools considered in US Dollars
    * `Solar Panel Install Cost (USD)` is the average cost of installing solar panels for the schools considered in US Dollars
